In [2]:
%pip install fastparquet

import pandas as pd

You should consider upgrading via the '/home/ramin/Developer/venv/salsa-evaluation/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Parameters
data = "../../salsa/data/single_partition/partition_0/left_index.gzip"
sample = 2000
min_interactions = 16
output = "../data/users.json"

In [4]:
# Import users from dataset
keep_columns = [0, 1]

df = pd.read_parquet(data, columns=['adjacency_list', 'user_id'])
df.reset_index(level=0, inplace=True)

In [5]:
df["len_adjacency_list"] = df["adjacency_list"].str.len()
active_users = df[df["len_adjacency_list"] >= min_interactions]

In [6]:
# Sample users
if sample:
    active_users = active_users.sample(n=sample)

active_users

,user_id,adjacency_list,len_adjacency_list
1016509,4760671340,"[1365101939126583296, 1364807212204511232, 136...",25
1019762,4817927613,"[1364975206150864896, 1365524779004923904, 136...",25
666459,1401185270,"[1364613338622222336, 1364582399837827072, 136...",41
2144879,1358225702999502859,"[1361385414628573184, 1361385022855532544, 136...",27
898247,3020900987,"[1365639321596944384, 1365602752563396608, 136...",30
...,...,...,...
1661653,1193528624042762241,"[1364907506984644608, 1365651156735725568, 136...",26
1460552,1070853360033689600,"[1361727060872482816, 1365188695687655424, 136...",28
1509235,1101636431959781376,"[1365065121962229760, 1364637562128789504, 136...",32
1231580,889939344668151808,"[1364932015729090560, 1364981695565324288, 136...",31


In [7]:
# Save results
active_users["user_id"].to_json(output, orient="table", index=False)
active_users.reset_index(drop=True)

,user_id,adjacency_list,len_adjacency_list
0,4760671340,"[1365101939126583296, 1364807212204511232, 136...",25
1,4817927613,"[1364975206150864896, 1365524779004923904, 136...",25
2,1401185270,"[1364613338622222336, 1364582399837827072, 136...",41
3,1358225702999502859,"[1361385414628573184, 1361385022855532544, 136...",27
4,3020900987,"[1365639321596944384, 1365602752563396608, 136...",30
...,...,...,...
1995,1193528624042762241,"[1364907506984644608, 1365651156735725568, 136...",26
1996,1070853360033689600,"[1361727060872482816, 1365188695687655424, 136...",28
1997,1101636431959781376,"[1365065121962229760, 1364637562128789504, 136...",32
1998,889939344668151808,"[1364932015729090560, 1364981695565324288, 136...",31
